In [1]:
from iit import generate_data
from model import TorchDeepNeuralClassifier
import numpy as np
import torch

In [2]:
vals = ['zero', 'one', 'two', 'three', 'four',
       'five', 'six', 'seven', 'eight', 'nine']

X, y = generate_data(vals, 10000)

In [3]:
X[:5]

[['seven', 'two', 'six'],
 ['six', 'one', 'one'],
 ['five', 'zero', 'six'],
 ['two', 'eight', 'four'],
 ['eight', 'one', 'two']]

In [4]:
y[:5]

[56, 12, 30, 24, 24]

Training a feed-forward network using randomized embeddings.

In [5]:
output_size = 163
num_inputs = 3
num_layers = 2
embed_dim = 15

mod = TorchDeepNeuralClassifier(vals, output_size, num_inputs, num_layers, embed_dim)

In [6]:
mod.fit(X, y)

Stopping after epoch 866. Training loss did not improve more than tol=1e-05. Final error is 0.0012900984002044424.

TorchDeepNeuralClassifier(
	batch_size=1028,
	max_iter=1000,
	eta=0.001,
	optimizer_class=<class 'torch.optim.adam.Adam'>,
	l2_strength=0,
	gradient_accumulation_steps=1,
	max_grad_norm=None,
	validation_fraction=0.1,
	early_stopping=False,
	n_iter_no_change=10,
	warm_start=False,
	tol=1e-05,
	hidden_dim=50,
	hidden_activation=Tanh(),
	num_layers=2)

In [7]:
X_test, y_test = generate_data(vals, 1000)

In [8]:
from sklearn.metrics import classification_report

preds = mod.predict(X_test)

print("\nClassification report:")
print(classification_report(y_test, preds))


Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       131
           1       1.00      1.00      1.00         4
           2       1.00      1.00      1.00         6
           3       1.00      1.00      1.00         9
           4       1.00      1.00      1.00        15
           5       1.00      1.00      1.00         9
           6       1.00      1.00      1.00        14
           7       1.00      1.00      1.00        15
           8       1.00      1.00      1.00        33
           9       1.00      1.00      1.00        20
          10       1.00      1.00      1.00        12
          11       1.00      1.00      1.00         5
          12       1.00      1.00      1.00        29
          13       1.00      1.00      1.00         6
          14       1.00      1.00      1.00        19
          15       1.00      1.00      1.00        17
          16       1.00      1.00      1.00        29
   

Training a feed-forward network using BERT embeddings.